# import

In [1]:
import pandas as pd
import json
import requests
from pathlib import Path
import numpy as np
import sys
import requests
import json
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib inline


# directories


In [2]:
if sys.platform =='linux':
    path_d = Path('/mnt/d')
else:
    path_d = Path('D://')
data_dir = path_d/'data'/'dpe_full'   

plot_dir = Path('.')/'plot'
plot_dir.mkdir(exist_ok=True,parents=True)

# read exemple

In [3]:
dept = '38'
td007 =pd.read_csv(data_dir/dept/'td007_paroi_opaque.csv',dtype=str)
td007 =pd.read_csv(data_dir/dept/'td007_paroi_opaque.csv',dtype=str)
td006 =pd.read_csv(data_dir/dept/'td006_batiment.csv',dtype=str)
td001 =pd.read_csv(data_dir/dept/'td001_dpe.csv',dtype=str)
td008 =pd.read_csv(data_dir/dept/'td008_baie.csv',dtype=str)


In [4]:
from assets_orm import DPEMetaData

In [5]:
meta = DPEMetaData()
self=meta

In [6]:
from td007_processing import merge_td007_tr_tv,postprocessing_td007
from td008_processing import merge_td008_tr_tv,postprocessing_td008
from td001_merge import merge_td001_dpe_id_envelope

In [7]:
td008 = merge_td008_tr_tv(td008)
td008 = postprocessing_td008(td008)

In [8]:
td007 = merge_td007_tr_tv(td007)
td007 = postprocessing_td007(td007,td008)

In [9]:
td007,td008 = merge_td001_dpe_id_envelope(td001=td001,td006=td006,td007=td007,td008=td008)


In [14]:
td007.columns

Index(['td007_paroi_opaque_id', 'td006_batiment_id',
       'tr014_type_parois_opaque_id', 'reference', 'deperdition_thermique',
       'tv001_coefficient_reduction_deperditions_id',
       'tv002_local_non_chauffe_id',
       'coefficient_transmission_thermique_paroi',
       'coefficient_transmission_thermique_paroi_non_isolee', 'tv003_umur_id',
       'tv004_umur0_id', 'tv005_upb_id', 'tv006_upb0_id', 'tv007_uph_id',
       'tv008_uph0_id', 'resistance_thermique_isolation',
       'epaisseur_isolation', 'surface_paroi', 'tr014_Sous-Type',
       'tr014_Est_efface', 'tv001_aiu_aue', 'tv001_aiu_aue_min',
       'tv001_aiu_aue_max', 'tv001_uv_ue', 'tv001_aue_isole',
       'tv001_aiu_isole', 'tv001_valeur', 'tv001_est_efface',
       'tv002_Type de Bâtiment', 'tv002_Local non chauffée', 'tv002_Uv,ue',
       'tv003_Mur Isolé', 'tv003_Année construction', 'tv003_Année isolation',
       'tv003_Zone Hiver', 'tv003_Effet Joule', 'tv003_Umur',
       'tv004_Matériaux', 'tv004_épaisseur', '

In [10]:
from utils import agg_pond_avg

In [11]:
td007_murs = td007.loc[td007.tr014_type_parois_opaque_id.isin(['TR014_002', 'TR014_001'])]
td007_pb = td007.loc[td007.tr014_type_parois_opaque_id == 'TR014_003']
td007_ph = td007.loc[td007.tr014_type_parois_opaque_id == 'TR014_004']

In [16]:
Umurs_ext_avg = agg_pond_avg(td007_murs, 'coefficient_transmission_thermique_paroi', 'surface_paroi_opaque_exterieur_infer',
                         'td001_dpe_id').to_frame('Umurs_ext_avg')

Umurs_avg = agg_pond_avg(td007_murs, 'coefficient_transmission_thermique_paroi', 'surface_paroi_deperditive_infer',
                         'td001_dpe_id').to_frame('Umurs_deper_avg')

D:\Cstbdata\MONWORKSPACE\Gitlab\de-dpe-processing\data_processing_cstb\traitement_donnees_metier\utils.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[pond_col] = table[pond]
D:\Cstbdata\MONWORKSPACE\Gitlab\de-dpe-processing\data_processing_cstb\traitement_donnees_metier\utils.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[pond_value_col_temp] = table[pond_col] * table[value_col]


KeyError: 'surface_paroi_opaque_exterieur_infer'

In [ ]:


Uplancher_avg = agg_pond_avg(td007_plancher, 'coefficient_transmission_thermique_paroi', 'surface_paroi_deperditive_infer',
                         'td001_dpe_id').to_frame('Uplanchers_deper_avg')

In [ ]:


Uplafond_avg = agg_pond_avg(td007_plafond, 'coefficient_transmission_thermique_paroi', 'surface_paroi_deperditive_infer',
                         'td001_dpe_id').to_frame('Uplafonds_deper_avg')